In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder

df = pd.read_csv('heart.csv')

# Приводим названия столбцов к нижнему регистру и убираем пробелы
df.columns = df.columns.str.strip().str.lower()

# Списки признаков по типу 
num_cols = ['age', 'cholesterol', 'restingbp', 'maxhr', 'oldpeak']
cat_cols = ['sex', 'chestpaintype', 'exerciseangina', 'restingecg', 'st_slope']

# Обработка пропущенных значений
for col in num_cols:
    if col in df.columns:
        if df[col].isnull().sum() > 0:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
    else:
        print(f"Внимание: числовой столбец '{col}' отсутствует в данных!")

for col in cat_cols:
    if col in df.columns:
        if df[col].isnull().sum() > 0:
            mode_val = df[col].mode()[0]
            df[col].fillna(mode_val, inplace=True)
    else:
        print(f"Внимание: категориальный столбец '{col}' отсутствует в данных!")

# Удаление выбросов по методу IQR
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print(f"{column}: удаляется {(data[column] < lower).sum() + (data[column] > upper).sum()} выбросов")
    return data[(data[column] >= lower) & (data[column] <= upper)]

for col in ['age', 'cholesterol', 'restingbp', 'maxhr']:
    if col in df.columns:
        df = remove_outliers_iqr(df, col)
    else:
        print(f"Внимание: столбец '{col}' отсутствует, пропускаем удаление выбросов по нему.")

# Масштабирование числовых признаков
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# Кодирование категориальных признаков
if 'sex' in df.columns:
    le = LabelEncoder()
    df['sex'] = le.fit_transform(df['sex'])
else:
    print("Внимание: столбец 'sex' отсутствует, кодирование пропущено.")

# One-hot кодирование для остальных категориальных признаков
to_onehot = [col for col in cat_cols if col != 'sex' and col in df.columns]
df = pd.get_dummies(df, columns=to_onehot, drop_first=True)

print("\nОбработка завершена. Итоговые данные:")
print(df.info())
print(df.head())

age: удаляется 0 выбросов
cholesterol: удаляется 183 выбросов
restingbp: удаляется 20 выбросов
maxhr: удаляется 0 выбросов

Обработка завершена. Итоговые данные:
<class 'pandas.core.frame.DataFrame'>
Index: 715 entries, 0 to 917
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   age                715 non-null    float64
 1   sex                715 non-null    int32  
 2   restingbp          715 non-null    float64
 3   cholesterol        715 non-null    float64
 4   fastingbs          715 non-null    int64  
 5   maxhr              715 non-null    float64
 6   oldpeak            715 non-null    float64
 7   heartdisease       715 non-null    int64  
 8   chestpaintype_ATA  715 non-null    bool   
 9   chestpaintype_NAP  715 non-null    bool   
 10  chestpaintype_TA   715 non-null    bool   
 11  exerciseangina_Y   715 non-null    bool   
 12  restingecg_Normal  715 non-null    bool   
 13  restingecg_ST